---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle
from operator import itemgetter
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MaxAbsScaler

#%matplotlib inline

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
#P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def plot_dist(degree_values, values_freq):
    
    plt.figure(figsize=(20,25))
    for ii in range(5):
        plt.subplot(5,1,ii+1)
        x = list(degree_values[ii])
        y = values_freq[ii]
        plt.bar(x, y)
        
    plt.show()
    return()


def graph_identification():
    
    # calculate the degree distribution for each network

    degree_dist = []
    degree_values =[]
    values_freq = []
    for ii in range(5):
        #print(set(P1_Graphs[ii].degree().values()))
        degree_dist.append(P1_Graphs[ii].degree().values())
        degree_values.append(set(P1_Graphs[ii].degree().values()))
        values_freq.append([list(degree_dist[ii]).count(jj) for jj in degree_values[ii]])
        
    #print(values_freq)
    #print(degree_values)
    #plot_dist(degree_values, values_freq)
    
    return ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']
    

graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagmentSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagmentSalary` indicates whether that person is receiving a managment position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a managment position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a managment salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a managment position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will receive full points.

Using your trained classifier, return a series of length 252 with the data being the probability of receiving managment salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [5]:
def plot_features(df_train):
    
    plt.figure(figsize=(20,5))
    
    plt.subplot(1,3,1)
    plt.scatter(df_train['Degree_Cent'], df_train['Close_Cent'], c=df_train['ManagementSalary'])
    
    plt.subplot(1,3,2)
    plt.scatter(df_train['Between_Cent'], df_train['Page_Rank'], c=df_train['ManagementSalary'])
    
    plt.subplot(1,3,3)
    plt.scatter(df_train['Hub_Score'], df_train['Auth_Score'], c=df_train['ManagementSalary'])
    
    plt.show()

In [6]:
def salary_predictions():
    
    # create dataframe of nodes
    df = pd.DataFrame(index=G.nodes())
    df['Dept'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    #print((nx.get_node_attributes(G, 'ManagementSalary')))
    #print(df)
    #print(G.nodes(data=True))
    #print(df.head(50))
    
    # create features for training, the selected features should be nodal features
    # Feature 1: Degree Centrality
    # Feature 2: Closeness Centrality
    # Feature 3: Betweenness Centrality
    # Feature 4: Page Rank with damping
    # Feature 5: HITS Hub Score
    # Feature 6: HITS Authority Score
    df['Degree_Cent'] = pd.Series(nx.degree_centrality(G))
    df['Close_Cent'] = pd.Series(nx.closeness_centrality(G))
    df['Between_Cent'] = pd.Series(nx.betweenness_centrality(G))
    df['Page_Rank'] = pd.Series(nx.pagerank(G))
    df['Hub_Score'] = pd.Series(nx.hits(G)[0])
    df['Auth_Score'] = pd.Series(nx.hits(G)[1])
    #print((nx.hits(G)))
    #print(pd.Series(nx.hits(G)[0]))
    #print(pd.Series(nx.hits(G)[1]))
    #print(pd.Series(nx.hits(G)[0]) - pd.Series(nx.hits(G)[1]))
    
    # split dataframe into 2
    #df_train = df[df['ManagementSalary'].notna()]
    #df_test = df[df['ManagementSalary'].isna()]
    df_train = df[(df['ManagementSalary'] == 0.0) | (df['ManagementSalary' ]== 1.0)]
    df_test = df[(df['ManagementSalary'] != 0.0) & (df['ManagementSalary'] != 1.0)]
    #X_notnumm = np.n
    #plot_features(df_train)
    #print(df_train.shape)
    #print(df_test.shape)
    
    # format data for classifier
    features = ['Degree_Cent', 'Close_Cent', 'Between_Cent', 'Page_Rank', 'Hub_Score', 'Auth_Score']
    X = df_train[features]
    y = df_train['ManagementSalary']
    X_test = df_test[features]
    #print(X.shape)
    #print(X_test.shape)
    #print(y.shape)
    
    X_train, X_cval, y_train, y_cval = train_test_split(X, y, random_state=1)

    #print(X_test)
    scaler = MaxAbsScaler()
    scaler.fit(X_train)
    X = scaler.transform(X)
    X_train = scaler.transform(X_train)
    X_cval = scaler.transform(X_cval)
    X_test = scaler.transform(X_test)
    #print(X_test)

    # Train classifier1
    svc_model = SVC(probability=True)
    svc_model.fit(X_train, y_train)
    #print(svc_model.score(X_train, y_train))
    #print(svc_model.score(X_cval, y_cval))
    #print(roc_auc_score(y_cval, svc_model.predict(X_cval)))
    svc_prediction = svc_model.predict_proba(X_test)
    #print(svc_prediction)

    # Train classifier2
    neighbour_model = KNeighborsClassifier()
    neighbour_model.fit(X_train, y_train)
    #print(neighbour_model.score(X_train, y_train))
    #print(neighbour_model.score(X_cval, y_cval))
    #print(roc_auc_score(y_cval, neighbour_model.predict(X_cval)))
    neighbour_prediction = neighbour_model.predict_proba(X_test)
    #print(neighbour_prediction)

    # Train classifier3
    forest_model = RandomForestClassifier()
    forest_model.fit(X_train, y_train)
    #print(forest_model.score(X_train, y_train))
    #print(forest_model.score(X_cval, y_cval))
    #print(roc_auc_score(y_cval, forest_model.predict(X_cval)))
    forest_prediction = forest_model.predict_proba(X_test)
    #print(forest_prediction)
    
    # Train classifier4
    logistic_model = LogisticRegression(C=100)
    logistic_model.fit(X_train, y_train)
    #print(neighbour_model.score(X_train, y_train))
    #print(neighbour_model.score(X_cval, y_cval))
    #print(roc_auc_score(y_cval, logistic_model.predict(X_cval)))
    logistic_prediction = logistic_model.predict_proba(X_test)
    #print(neighbour_prediction)

    
    final_model = RandomForestClassifier()
    final_model.fit(X, y)
    prediction = final_model.predict_proba(X_test)
    result = pd.Series(prediction[:,1], index=df_test.index)

    return result

salary_predictions()

1       0.1
2       1.0
5       1.0
8       0.1
14      0.1
18      0.1
27      0.1
30      0.1
31      0.2
34      0.1
37      0.0
40      0.2
45      0.1
54      0.2
55      0.1
60      0.2
62      1.0
65      1.0
77      0.0
79      0.3
97      0.0
101     0.0
103     0.0
108     0.1
113     0.1
122     0.1
141     0.3
142     1.0
144     0.0
145     0.0
       ... 
913     0.0
914     0.0
915     0.0
918     0.0
923     0.0
926     0.0
931     0.0
934     0.0
939     0.0
944     0.0
945     0.0
947     0.2
950     0.4
951     0.0
953     0.0
959     0.0
962     0.0
963     0.3
968     0.0
969     0.0
974     0.0
984     0.0
987     0.0
989     0.0
991     0.0
992     0.0
994     0.0
996     0.0
1000    0.0
1001    0.0
Length: 252, dtype: float64

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [7]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
#future_connections.head(10)
#print(future_connections)

Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will receive full points.

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [8]:
def new_connections_predictions():
    
    # create graph 
    #G2 = nx.Graph()
    #G2.add_edges_from([[x[0], x[1]] for x in future_connections.index])
    #print(type([[x[0], x[1]] for x in future_connections.index]))
    #attrib_index = future_connections.index.values
    #attrib_value = future_connections[['Future Connection']].values[:,0]
    #attrib_dict = dict(zip(attrib_index, attrib_value))
    #print((attrib_value.shape))
    #print((attrib_index.shape))
    #print(attrib_dict)
    #nx.set_edge_attributes(G2, 'future_connection', attrib_dict)
    
    # create and edges dataframe 
    df = pd.DataFrame(index = nx.non_edges(G))
    #df2 = pd.DataFrame(index=G2.edges())
    #df2['Future_Connections'] = pd.Series(nx.get_edge_attributes(G2, 'future_connection'))
    #print(df.tail())

    #create edge features
    df['Pref_Attach'] = [i[2] for i in nx.preferential_attachment(G, df.index)]
    #df['Adamic'] = [i[2] for i in nx.adamic_adar_index(G, df.index)]
    df['Jacard'] = [i[2] for i in nx.jaccard_coefficient(G, df.index)]
    df['Neighbour'] = df.index.map(lambda edge: len(list(nx.common_neighbors(G, edge[0], edge[1]))))
    #print(df2.tail(100))

    df = df.join(future_connections, how='inner')
    #print(df.head())
    #print(df.shape)
    
    #create edge features
    
    # split data set into data with future predictions and those without
    df_Con = df[(df['Future Connection']== 0) | (df['Future Connection']== 1)]
    df_noCon = df[(df['Future Connection'] != 0) & (df['Future Connection'] != 1)]
    #print(df2_Con.shape)
    #print(df2_noCon.shape)

    # format data for classifier
    features = ['Pref_Attach', 'Jacard', 'Neighbour']
    X = df_Con[features]
    y = df_Con['Future Connection']
    X_test = df_noCon[features]
    #print(X.shape)
    #print(y.shape)
    
    X_train, X_cval, y_train, y_cval = train_test_split(X, y)
    
    #print(X_test)
    scaler = MaxAbsScaler()
    scaler.fit(X_train)
    X = scaler.transform(X)
    X_train = scaler.transform(X_train)
    X_cval = scaler.transform(X_cval)
    X_test = scaler.transform(X_test)
    #print(X_test)

    # Train classifier2
    neighbour_model = KNeighborsClassifier(n_neighbors=50)
    neighbour_model.fit(X_train, y_train)
    #print(neighbour_model.score(X_train, y_train))
    #print(neighbour_model.score(X_cval, y_cval))
    #print(roc_auc_score(y_cval, neighbour_model.predict(X_cval)))
    neighbour_prediction = neighbour_model.predict_proba(X_test)
    #print(neighbour_prediction)

    # Train classifier2
    logistic_model = LogisticRegression()
    logistic_model.fit(X_train, y_train)
    #print(neighbour_model.score(X_train, y_train))
    #print(neighbour_model.score(X_cval, y_cval))
    #print(roc_auc_score(y_cval, logistic_model.predict(X_cval)))
    logistic_prediction = logistic_model.predict_proba(X_test)
    #print(neighbour_prediction)

    # Train classifier3
    forest_model = RandomForestClassifier(n_estimators=50)
    forest_model.fit(X_train, y_train)
    #print(forest_model.score(X_train, y_train))
    #print(forest_model.score(X_cval, y_cval))
    #print(roc_auc_score(y_cval, forest_model.predict(X_cval)))
    forest_prediction = forest_model.predict_proba(X_test)
    #print(forest_prediction)
    
    final_model = RandomForestClassifier()
    final_model.fit(X, y)
    prediction = final_model.predict_proba(X_test)
    result = pd.Series(prediction[:,1], index=df_noCon.index)
    
    return result

new_connections_predictions()

(0, 9)          0.000000
(0, 19)         0.000000
(0, 20)         0.000000
(0, 35)         0.000000
(0, 38)         0.000000
(0, 42)         0.700000
(0, 43)         0.000000
(0, 50)         0.000000
(0, 53)         0.800000
(0, 54)         0.404167
(0, 62)         0.000000
(0, 63)         0.000000
(0, 69)         0.000000
(0, 72)         0.013049
(0, 83)         0.000000
(0, 90)         0.042866
(0, 91)         0.000000
(0, 95)         0.000000
(0, 100)        0.000000
(0, 106)        1.000000
(0, 108)        0.000000
(0, 109)        0.016947
(0, 110)        0.014989
(0, 118)        0.000000
(0, 122)        0.000000
(0, 131)        0.000000
(0, 133)        1.000000
(0, 140)        0.014286
(0, 149)        0.591667
(0, 151)        0.000000
                  ...   
(988, 989)      0.008371
(988, 996)      0.015322
(988, 997)      0.142899
(988, 1000)     0.016965
(988, 1002)     0.015322
(989, 994)      0.012345
(989, 996)      0.012345
(989, 1003)     0.012345
(989, 1004)     0.012345
